Name Cleaning

I have 4 lists of data
- full_1_list - All names in table 1
- full_2_list - All names in table 2
- names_only_in_1 = Names that are uniqure to table 1
- names_only_in_2 = Names that are uniqure to table 2

I will compare 
- names_only_in_1 to names_only_in_2
- names_only_in_2 to names_only_in_1
- names_only_in_1 to full_2_list
- names_only_in_2 to full_1_list

In [8]:
#Libraries
%store -r
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import json
import requests, bs4, os, sys
from unidecode import unidecode
from fuzzywuzzy import fuzz
from collections import defaultdict

In [9]:
#Import data from data_collection and cleaning and rename for testing
df_full_FPL_test = df_full_FPL
df_all_test = df_all

1. names_only_in_1 to names_only_in_2

In [14]:
#Check which names don't match in each list

# Get the sets of names in each dataframe column
set1 = set(df_full_FPL_test["player_name"])
set2 = set(df_all_test['player_name'])
full_1_list = list(set1)
full_2_list = list(set2)

# Get the names that are in set1 but not in set2 and get the names that are in set2 but not in set1

names_only_in_1 = list(set1 - set2)
names_only_in_2 = list(set2 - set1)

# Print the results
print('Names only in set1:', names_only_in_1)
print('Names only in set2:', names_only_in_2)

Names only in set1: ['javier manquillo gaitan', 'wesley moraes', 'daniel castelo podence', 'nelson cabral semedo', 'alisson ramses becker', 'mathias jorgensen', 'andre-frank zambo anguissa', 'juan camilo hernandez suarez', 'bamidele alli', 'ahmed el mohamady', 'emerson leite de souza junior', 'ricardo barbosa pereira', 'felipe augusto de almeida monteiro', 'bruno guimaraes rodriguez moura', 'hwang hee-chan', 'willian jose da silva', 'marcus oliveira alencar', 'norberto murara neto', 'pape matar sarr', 'jonathan castro otto', 'facundo pellistri rebollo', 'matheus santos carneiro da cunha', 'diogo teixeira da silva', 'miguel almiron rejala', 'bobby decordova-reid', 'david luiz moreira marinho', 'diogo dalot teixeira', 'junior firpo adames', 'alejandro garnacho ferreyra', 'diego da silva costa', 'ruben gato alves dias', 'mathew ryan', 'pedro lomba neto', 'oghenekaro peter etebo', 'gabriel teodoro martinelli silva', 'darwin nunez ribeiro', 'pervis estupinan', 'kepa arrizabalaga', 'fabio fr

In [15]:
#Check all matches and check if any of these matches appear more than once

all_matches_1 = []

# Initialize a dictionary to keep track of the number of times each name in names_only_in_1 has been matched
match_counts_1 = defaultdict(int)

# Match the names in names_only_in_1 to names in names_only_in_2
for name1 in names_only_in_1:
    # Find the best match in list2 using the fuzzywuzzy token set ratio
    best_match_1 = max(names_only_in_2, key=lambda name2: fuzz.token_set_ratio(name1, name2))
    # If the match score is above a certain threshold, consider the names to match
    if fuzz.token_set_ratio(name1, best_match_1) > 90:
        #Add matches to all_matches list
        all_matches_1.append((name1, best_match_1))
        # Increment the match count for the best match
        match_counts_1[best_match_1] += 1

# Initialize a list to store the matching names where match[i] has been matched with more than once
double_matches_1 = []

# Store the matching names where match[i] has been matched with more than once
for name2, count in match_counts_1.items():
    if count > 1:
        # Find all the names in list1 that match the current name2
        matches_1 = [(name1, name2) for name1 in names_only_in_1 if fuzz.token_set_ratio(name1, name2) > 90]
        # Add the matching names to the matches_2 list
        double_matches_1.extend(matches_1)

print(all_matches_1)
print(double_matches_1)


[('wesley moraes', 'wesley'), ('daniel castelo podence', 'daniel podence'), ('nelson cabral semedo', 'nelson semedo'), ('alisson ramses becker', 'alisson'), ('juan camilo hernandez suarez', 'juan camilo hernandez'), ('ahmed el mohamady', 'ahmed elmohamady'), ('emerson leite de souza junior', 'emerson'), ('ricardo barbosa pereira', 'ricardo pereira'), ('felipe augusto de almeida monteiro', 'felipe'), ('bruno guimaraes rodriguez moura', 'bruno guimaraes'), ('willian jose da silva', 'willian'), ('norberto murara neto', 'neto'), ('pape matar sarr', 'pape sarr'), ('facundo pellistri rebollo', 'facundo pellistri'), ('matheus santos carneiro da cunha', 'matheus cunha'), ('bobby decordova-reid', 'bobby reid'), ('david luiz moreira marinho', 'david luiz'), ('diogo dalot teixeira', 'diogo dalot'), ('junior firpo adames', 'junior firpo'), ('diego da silva costa', 'diego costa'), ('ruben gato alves dias', 'ruben dias'), ('pedro lomba neto', 'pedro neto'), ('oghenekaro peter etebo', 'oghenekaro ete

In [ ]:
#Remove double matches from list for further analysis

# Compute the set difference to remove double_matches_1 from all_matches_1. Then turn back to a list.
single_matches_1 = list(set(all_matches_1) - set(double_matches_1))

# Print the tuples in the new matches list
for match in single_matches_1:
   print(match)

In [13]:
#On a sense check only 1 name gives an issue 'Bernard Anicio Caldeira Duarte', 'Bernard'
#Bernard Anicio Caldeira Duarte played at everton but in Understat we have multiple bernards. 
# The Bernard in question is the player with id 7063 so I will edit the name in Understat and run again.
df_all_test.loc[df_all_test['player_id'] == 7063, 'player_name'] = 'Bernard Anicio Caldeira Duarte'

In [18]:
#Now I am confident these names match I will update the FPL table with the Understat name

for match in single_matches_1:
    old_name, new_name = match
    df_full_FPL_test['player_name'] = df_full_FPL_test['player_name'].str.replace(old_name, new_name)

In [20]:
#Now for the the names that are matching more than once. I need to export and run a manual check.
#There are 41

# Convert the list of tuples to a Pandas DataFrame
df_double_matches_1 = pd.DataFrame(double_matches_1, columns=["FPL Name", "Understat Name"])

# Export the DataFrame to a CSV file
df_double_matches_1.to_csv("df_double_matches_1.csv", index=False)

In [27]:
#On a sense check only 1 name gives an issue 'Emerson Palmieri dos Santos', 'Emerson'
# The Emerson in question is the player with id 1245 so I will edit the name in Understat and run again.
df_all_test.loc[df_all_test['player_id'] == 1245, 'player_name'] = 'emerson palmieri dos santos'

In [24]:
#After manually checking list, import the correct list and update FPL list of names

df_double_matches_1_corrected = pd.read_csv('/Users/jon/Documents/fpl_points_prediction/df_double_matches_1_corrected.csv', delimiter=";", index_col=False)
df_double_matches_1_corrected = list(df_double_matches_1_corrected.to_records(index=False))
for match in df_double_matches_1_corrected:
    old_name, new_name = match
    df_full_FPL_test['player_name'] = df_full_FPL_test['player_name'].str.replace(old_name, new_name)

2. names_only_in_2 to names_only_in_1
I will now run this process again the other way

In [28]:
#First Update lists to account for current changes

# Get the sets of names in each dataframe column
set1 = set(df_full_FPL_test["player_name"])
set2 = set(df_all_test['player_name'])
full_1_list = list(set1)
full_2_list = list(set2)

# Get the names that are in set1 but not in set2 and get the names that are in set2 but not in set1

names_only_in_1 = list(set1 - set2)
names_only_in_2 = list(set2 - set1)

# Print the results
print('Names only in set1:', names_only_in_1)
print('Names only in set2:', names_only_in_2)

Names only in set1: ['javier manquillo gaitan', 'moises caicedo corozo', 'mahmoud ahmed ibrahim hassan', 'vitor ferreira', 'mohamed naser el sayed elneny', 'frederico rodrigues de paula santos', 'hector junior firpo', 'oluwasemilogo adesewo ibidapo ajayi', 'lucas tolentino coelho de lima', 'tino livramento', 'mathias jorgensen', 'mateus cardoso lemos martins', 'andre-frank zambo anguissa', 'bamidele alli', 'fabio henrique tavares', 'fernando luiz rosa', 'francisco casilla cortes', 'solomon march', 'edward nketiah', 'adama traore diarra', 'hwang hee-chan', 'joseph willock', 'pelenda joshua dasilva', 'francisco femenia far', 'francisco jorge tomas oliveira', 'rayan ait-nouri', 'sergi canos tenes', 'marcus oliveira alencar', 'jonathan castro otto', 'pablo fornals malla', 'diogo teixeira da silva', 'cedric alves soares', 'miguel almiron rejala', 'emiliano martinez romero', 'heung-min son', 'jeremy sarmiento morante', 'benjamin white', 'nayef aguerd', 'alejandro garnacho ferreyra', 'matty c

In [29]:
#Again Check all matches and check if any of these matches appear more than once

all_matches_2 = []

# Initialize a dictionary to keep track of the number of times each name in names_only_in_2 has been matched
match_counts_2 = defaultdict(int)

# Match the names in names_only_in_1 to names in names_only_in_2
for name1 in names_only_in_2:
    # Find the best match in list2 using the fuzzywuzzy token set ratio
    best_match_2 = max(names_only_in_1, key=lambda name2: fuzz.token_set_ratio(name1, name2))
    # If the match score is above a certain threshold, consider the names to match
    if fuzz.token_set_ratio(name1, best_match_2) > 90:
        #Add matches to all_matches list
        all_matches_2.append((name1, best_match_2))
        # Increment the match count for the best match
        match_counts_2[best_match_2] += 1

# Initialize a list to store the matching names where match[i] has been matched with more than once
double_matches_2 = []

# Store the matching names where match[i] has been matched with more than once
for name2, count in match_counts_2.items():
    if count > 1:
        # Find all the names in list that match the current name
        matches_2 = [(name1, name2) for name1 in names_only_in_2 if fuzz.token_set_ratio(name1, name2) > 90]
        # Add the matching names to the double_matches_2 list
        double_matches_2.extend(matches_2)

print(all_matches_2)
print(double_matches_2)

#Empty as expected.

[]
[]


3. - names_only_in_1 to full_2_list

#No need to update lists again as no further chnages made.


In [30]:
all_matches_3 = []

# Initialize a dictionary to keep track of the number of times each name in names_only_in_1 has been matched
match_counts_3 = defaultdict(int)

# Match the names in names_only_in_1 to names in full_2_list
for name1 in names_only_in_1:
    # Find the best match in list2 using the fuzzywuzzy token set ratio
    best_match_3 = max(full_2_list, key=lambda name2: fuzz.token_set_ratio(name1, name2))
    # If the match score is above a certain threshold, consider the names to match
    if fuzz.token_set_ratio(name1, best_match_3) > 90:
        #Add matches to all_matches list
        all_matches_3.append((name1, best_match_3))
        # Increment the match count for the best match
        match_counts_3[best_match_3] += 1

# Initialize a list to store the matching names where match[i] has been matched with more than once
double_matches_3 = []

# Store the matching names where match[i] has been matched with more than once
for name2, count in match_counts_3.items():
    if count > 1:
        # Find all the names in list that match the current name
        matches_3 = [(name1, name2) for name1 in names_only_in_1 if fuzz.token_set_ratio(name1, name2) > 90]
        # Add the matching names to the double_matches_3 list
        double_matches_3.extend(matches_3)

print(all_matches_3)
print(double_matches_3)
#No double matches

[('javier manquillo gaitan', 'javier manquillo'), ('moises caicedo corozo', 'moises caicedo'), ('mohamed naser el sayed elneny', 'mohamed elneny'), ('hector junior firpo', 'junior firpo'), ('adama traore diarra', 'adama traore'), ('hwang hee-chan', 'hee-chan hwang'), ('rayan ait-nouri', 'rayan ait nouri'), ('sergi canos tenes', 'sergi canos'), ('pablo fornals malla', 'pablo fornals'), ('cedric alves soares', 'cedric soares'), ('miguel almiron rejala', 'miguel almiron'), ('emiliano martinez romero', 'emiliano martinez'), ('heung-min son', 'son heung-min'), ('jeremy sarmiento morante', 'jeremy sarmiento'), ('alejandro garnacho ferreyra', 'alejandro garnacho'), ('marc cucurella saseta', 'marc cucurella'), ('david de gea quintana', 'david de gea'), ('fabio freitas gouveia carvalho', 'fabio carvalho')]
[]


In [32]:
#After sense check, everything looks ok. So I will go ahead and update the FPL list
for match in all_matches_3:
    old_name, new_name = match
    df_full_FPL_test['player_name'] = df_full_FPL_test['player_name'].str.replace(old_name, new_name)

4. - names_only_in_2 to full_1_list

In [62]:
#First Update lists to account for current changes

# Get the sets of names in each dataframe column
set1 = set(df_full_FPL_test["player_name"])
set2 = set(df_all_test['player_name'])
full_1_list = list(set1)
full_2_list = list(set2)

# Get the names that are in set1 but not in set2 and get the names that are in set2 but not in set1

names_only_in_1 = list(set1 - set2)
names_only_in_2 = list(set2 - set1)

# Print the results
print('Names only in set1:', names_only_in_1)
print('Names only in set2:', names_only_in_2)

Names only in set1: ['mahmoud ahmed ibrahim hassan', 'vitor ferreira', 'frederico rodrigues de paula santos', 'oluwasemilogo adesewo ibidapo ajayi', 'lucas tolentino coelho de lima', 'tino livramento', 'mathias jorgensen', 'mateus cardoso lemos martins', 'andre-frank zambo anguissa', 'bamidele alli', 'fabio henrique tavares', 'fernando luiz rosa', 'francisco casilla cortes', 'solomon march', 'edward nketiah', 'joseph willock', 'pelenda joshua dasilva', 'francisco femenia far', 'francisco jorge tomas oliveira', 'marcus oliveira alencar', 'diogo teixeira da silva', 'benjamin white', 'nayef aguerd', 'matty cash', 'raphael dias belloli', 'carlos ribeiro dias', 'mathew ryan', 'alexandre moreno lopera', 'conrad egan-riley', 'jorge luiz frello filho', 'benjamin chilwell', 'nicolas nkoulou', 'daniel ceballos fernandez', 'carlos henrique casimiro']
Names only in set2: ['casemiro', 'alex moreno', 'cafu', 'cj egan-riley', "nicolas n'koulou", 'zanka', 'dani ceballos', 'mat ryan', 'willian jose da 

In [63]:
all_matches_4 = []

# Initialize a dictionary to keep track of the number of times each name in names_only_in_1 has been matched
match_counts_4 = defaultdict(int)

# Match the names in names_only_in_2 to names in full_1_list
for name1 in names_only_in_2:
    # Find the best match in list using the fuzzywuzzy token set ratio
    best_match_4 = max(full_1_list, key=lambda name2: fuzz.token_set_ratio(name1, name2))
    # If the match score is above a certain threshold, consider the names to match
    if fuzz.token_set_ratio(name1, best_match_4) > 80: #Lowering to 80 since not many were found
        #Add matches to all_matches list
        all_matches_4.append((name1, best_match_4))
        # Increment the match count for the best match
        match_counts_4[best_match_4] += 1

# Initialize a list to store the matching names where match[i] has been matched with more than once
double_matches_4 = []

# Store the matching names where match[i] has been matched with more than once
for name2, count in match_counts_4.items():
    if count > 1:
        # Find all the names in list that match the current name
        matches_4 = [(name1, name2) for name1 in names_only_in_2 if fuzz.token_set_ratio(name1, name2) > 90]
        # Add the matching names to the double_matches_4 list
        double_matches_4.extend(matches_4)

print(all_matches_4)
print(double_matches_4)
#Do double matches

[('cj egan-riley', 'conrad egan-riley'), ("nicolas n'koulou", 'nicolas nkoulou'), ('mat ryan', 'mathew ryan'), ('willian jose da silva', 'willian'), ('valentino livramento', 'tino livramento'), ('tete', 'kenny tete'), ('naif aguerd', 'nayef aguerd')]
[]


In [64]:
all_matches_4

[('cj egan-riley', 'conrad egan-riley'),
 ("nicolas n'koulou", 'nicolas nkoulou'),
 ('mat ryan', 'mathew ryan'),
 ('willian jose da silva', 'willian'),
 ('valentino livramento', 'tino livramento'),
 ('tete', 'kenny tete'),
 ('naif aguerd', 'nayef aguerd')]

In [61]:
#On a sense check only 2 names give issues
df_all_test.loc[df_all_test['player_id'] == 2280, 'player_name'] = 'jonathan castro otto'
df_all_test.loc[df_all_test['player_id'] == 2361, 'player_name'] = 'willian jose da silva' 

In [65]:
#After sense check, everything looks ok. So I will go ahead and update the FPL list
for match in all_matches_4:
    old_name, new_name = match
    df_full_FPL_test['player_name'] = df_full_FPL_test['player_name'].str.replace(old_name, new_name)

5. #Now I will look at the remianing unique values to make them match

In [68]:
#First Update lists to account for current changes

# Get the sets of names in each dataframe column
set1 = set(df_full_FPL_test["player_name"])
set2 = set(df_all_test['player_name'])
full_1_list = list(set1)
full_2_list = list(set2)

# Get the names that are in set1 but not in set2 and get the names that are in set2 but not in set1

names_only_in_1 = list(set1 - set2)
names_only_in_2 = list(set2 - set1)

# Print the results
print('Names only in set1:', names_only_in_1)
print('Names only in set2:', names_only_in_2)

Names only in set1: ['mahmoud ahmed ibrahim hassan', 'vitor ferreira', 'frederico rodrigues de paula santos', 'oluwasemilogo adesewo ibidapo ajayi', 'lucas tolentino coelho de lima', 'tino livramento', 'mathias jorgensen', 'mateus cardoso lemos martins', 'andre-frank zambo anguissa', 'bamidele alli', 'fabio henrique tavares', 'fernando luiz rosa', 'francisco casilla cortes', 'solomon march', 'edward nketiah', 'joseph willock', 'pelenda joshua dasilva', 'francisco femenia far', 'francisco jorge tomas oliveira', 'marcus oliveira alencar', 'diogo teixeira da silva', 'benjamin white', 'nayef aguerd', 'matty cash', 'raphael dias belloli', 'carlos ribeiro dias', 'mathew ryan', 'alexandre moreno lopera', 'conrad egan-riley', 'jorge luiz frello filho', 'kenny kenny tete', 'benjamin chilwell', 'nicolas nkoulou', 'daniel ceballos fernandez', 'carlos henrique casimiro']
Names only in set2: ['casemiro', 'alex moreno', 'cafu', 'cj egan-riley', "nicolas n'koulou", 'zanka', 'dani ceballos', 'mat ryan

In [69]:
#I need to export the first and run a manual check.

# Convert the list of tuples to a Pandas Series
df_unique_values_5 = pd.DataFrame(names_only_in_1, columns=["FPL Name"])

# Export the DataFrame to a CSV file
df_unique_values_5.to_csv("df_unique_values_5.csv", index=False)

In [73]:
#After manually checking list, import the correct list and update FPL list of names

df_df_unique_values_5_corrrected = pd.read_csv('/Users/jon/Documents/fpl_points_prediction/df_unique_values_5_corrrected.csv', delimiter=";", index_col=False)
df_df_unique_values_5_corrrected = list(df_df_unique_values_5_corrrected.to_records(index=False))
for match in df_df_unique_values_5_corrrected:
    old_name, new_name = match
    df_full_FPL_test['player_name'] = df_full_FPL_test['player_name'].str.replace(old_name, new_name)

6. #Now I will look at the remianing unique values to make them match

In [87]:
#First Update lists to account for current changes

# Get the sets of names in each dataframe column
set1 = set(df_full_FPL_test["player_name"])
set2 = set(df_all_test['player_name'])
full_1_list = list(set1)
full_2_list = list(set2)

# Get the names that are in set1 but not in set2 and get the names that are in set2 but not in set1

names_only_in_1 = list(set1 - set2)
names_only_in_2 = list(set2 - set1)

# Print the results
print('Names only in set1:', names_only_in_1)
print('Names only in set2:', names_only_in_2)

Names only in set1: []
Names only in set2: []


In [86]:
#Replacing names tat still don't match manually
df_full_FPL_test.loc[df_full_FPL_test['player_name'] == 'kiko femenía', 'player_name'] = 'kiko femenia'
df_full_FPL_test.loc[df_full_FPL_test['player_name'] == "solomon march", 'player_name'] = 'solly march'
df_full_FPL_test.loc[(df_full_FPL_test['team'] == 'Wolves') & (df_full_FPL_test['player_name'] == 'willian'), 'player_name'] = 'willian jose da silva'


In [88]:
#Check that lists of names are compatible
print(len(set(df_all_test["player_name"])))
print(len(set(df_full_FPL_test["player_name"])))

887
887


In [106]:
%store df_all_test
%store df_full_FPL_test

Stored 'df_all_test' (DataFrame)
Stored 'df_full_FPL_test' (DataFrame)
